In [1]:
import ROOT
import json
%jsroot on

Welcome to JupyROOT 6.22/06


# Load signal MC sample

In [2]:
#JobPath = input("Enter the path of signal MC sample: ") # JetAnalysis/Jet+btagging
JobPath = "JetAnalysis/NtupleAnalysis"
MCPath = "~/Work/Hbb/{}/Samples".format(JobPath)
f = ROOT.TFile("{}/ZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8.root".format(MCPath), "READ")
hist = ROOT.TH1F()
hist = f.Get("h_RecDiJet_Match_M")

# Scale the invariant mass spectra

In [3]:
MCInfoList = []
SigSampleName = "ZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8"
with open("../Database/MCInfo.json") as MCInfo:
    MCInfoList = json.load(MCInfo)
for i in MCInfoList:
    for key, value in i.items():
        if key == 'primary_name':
            if value == SigSampleName:
                factor = i['factor_2018']
                print("The primary name: ", value)
                print("The factor is ", factor)
                break
factor = factor * 0.004536324 # A correction to control trigger HLT_IsoMu20_v*
hist.Scale(factor)

The primary name:  ZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8
The factor is  0.0005726334482615378


# Set Roofit

In [4]:
# set RooFit variables
x = ROOT.RooRealVar("x","m_{dijet}",50, 200)
m = ROOT.RooRealVar("m","m",114, 110, 120) # mean
s = ROOT.RooRealVar("s","s",18.29787234, 5, ROOT.RooNumber.infinity()) # width or standard deviation

# for crystal ball function
a = ROOT.RooRealVar("a","a",1.55502e+00, -3, 10)
n = ROOT.RooRealVar("n","n",4.41255e+00, 0, 10)

# for Voigtian function
sv = ROOT.RooRealVar("sv","sv",0,0,2) # sigma in Voigtian

# for Bukin function
xi = ROOT.RooRealVar("xi","peak asymmetry",-1.8322e-01, -1, 1)
rho1 = ROOT.RooRealVar("rho1","left tail",-0.1790206614531008, -1, 0)
rho2 = ROOT.RooRealVar("rho2","right tail",0.21758760999527632, 0, 1)

# load the histogram
ral = ROOT.RooArgList(x) # this line must be added in pyroot
h = ROOT.RooDataHist("h","Mass of dijet",ral,ROOT.RooFit.Import(hist))

# set fit function 
#func = ROOT.RooCBShape("func","func",x,m,s,a,n) # crystal ball function
#func = ROOT.RooGaussian("func","func",x,m,s)
#func = ROOT.RooBreitWigner("func","func",x,m,s)
#func = ROOT.RooVoigtian("func","func",x,m,s,sv)
func = ROOT.RooBukinPdf("func","func",x,m,s,xi,rho1,rho2)


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

[#0] WARNING:InputArguments -- The parameter 'rho1' with range [-1, 0] of the RooBukinPdf 'func' exceeds the safe range of (-1, 0). Advise to limit its range.
[#0] WARNING:InputArguments -- The parameter 'rho2' with range [0, 1] of the RooBukinPdf 'func' exceeds the safe range of (0, 1). Advise to limit its range.
[#0] WARNING:InputArguments -- The parameter 'xi' with range [-1, 1] of the RooBukinPdf 'func' exceeds the safe range of (-1, 1). Advise to limit its range.


# Fit

In [5]:
# fit
massFit = func.fitTo(h, ROOT.RooFit.Save())
#func.fitTo(h,ROOT.RooFit.Range(50,160)) # fit in peak range
massFit.Print()

c = ROOT.TCanvas("c","c",800,600)

xframe1 = x.frame()

h.plotOn(xframe1)
func.plotOn(xframe1, ROOT.RooFit.LineColor(ROOT.kBlue))

h_pull = ROOT.RooHist()
h_pull = xframe1.pullHist()
xframe2 = x.frame()
xframe2.addObject(h_pull)

[#0] WARNING:InputArguments -- RooAbsPdf::fitTo(func) WARNING: a likelihood fit is requested of what appears to be weighted data.
       While the estimated values of the parameters will always be calculated taking the weights into account,
       there are multiple ways to estimate the errors of the parameters. You are advised to make an 
       explicit choice for the error calculation:
           - Either provide SumW2Error(true), to calculate a sum-of-weights-corrected HESSE error matrix
             (error will be proportional to the number of events in MC).
           - Or provide SumW2Error(false), to return errors from original HESSE error matrix
             (which will be proportional to the sum of the weights, i.e., a dataset with <sum of weights> events).
           - Or provide AsymptoticError(true), to use the asymptotically correct expression
             (for details see https://arxiv.org/abs/1911.01303).
[#1] INFO:NumericIntegration -- RooRealIntegral::init(func_Int[x]

Warning in <TMinuitMinimizer::SetLowerLimitedVariable>: not implemented - use as upper limit 1.E7 instead of +inf


# Change the style of TCanvas

In [34]:
upperPad = ROOT.TPad("upperPad", "upperPad", 0., 0.3, 1. , 1.)
lowerPad = ROOT.TPad("lowerPad", "lowerPad", 0., 0., 1. , 0.3)
upperPad.Draw()
lowerPad.Draw()
# upper pad
upperPad.cd()
upperPad.SetBottomMargin(0.)
xframe1.SetTitle("")
sigLine1 = ROOT.TLine(90, 0, 90, 0.03)
sigLine2 = ROOT.TLine(150, 0, 150, 0.03)
sigLine1.SetLineStyle(2)
sigLine2.SetLineStyle(2)
xframe1.Draw()
sigLine1.Draw()
sigLine2.Draw()
# lower pad
lowerPad.cd()
lowerPad.SetTopMargin(0.)
lowerPad.SetBottomMargin(0.3)
xframe2.SetTitle("")
xframe2.SetMaximum(5.)
xframe2.SetMinimum(-5.)
xframe2.GetYaxis().SetTitleSize(0.1)
xframe2.GetYaxis().SetNdivisions(505)
xframe2.GetYaxis().SetTitle("Pull")
xframe2.GetYaxis().SetTitleOffset(0.2)
xframe2.GetYaxis().SetLabelSize(0.08)
xframe2.GetXaxis().SetTitleSize(0.1)
xframe2.GetXaxis().SetLabelSize(0.08)

line0 = ROOT.TLine(50, 0, 200, 0)
line1 = ROOT.TLine(50, 1, 200, 1)
line2 = ROOT.TLine(50, -1, 200, -1)
line0.SetLineWidth(2)
line0.SetLineStyle(2)
line1.SetLineWidth(2)
line1.SetLineStyle(2)
line2.SetLineWidth(2)
line2.SetLineStyle(2)


xframe2.Draw()
line0.Draw()
line1.Draw()
line2.Draw()

In [10]:
xframe1.SetTitle("")
sigLine1 = ROOT.TLine(90, 0, 90, 0.029)
sigLine2 = ROOT.TLine(150, 0, 150, 0.029)
sigLine1.SetLineStyle(2)
sigLine1.SetLineWidth(2)
sigLine2.SetLineStyle(2)
sigLine2.SetLineWidth(2)
xframe1.Draw()
sigLine1.Draw()
sigLine2.Draw()

# Plot the figure

In [11]:
c.Draw()
m.Print()
#chi2 = ROOT.RooChi2Var("chi2","chi2",func,h)
chi2 = xframe1.chiSquare(5)
print("Chi-square: ",chi2)
c.SaveAs("fit.pdf")

Chi-square:  7.717382912354021
RooRealVar::m = 115.039 +/- 7.47389  L(110 - 120) 


Info in <TCanvas::Print>: pdf file fit.pdf has been created


# Automatically adjust parameters in the Bukin function (test)

In [ ]:
#massFit.Print()
#massFit.status()
count = 0
bukinDict = {"m": 0, "s": 0, "xi": 0, "rho1": 0, "rho2": 0}
while (massFit.status() != 0 or count < 100):
    bukinDict["m"] = m.getVal()
    bukinDict["s"] = s.getVal()
    bukinDict["xi"] = xi.getVal()
    bukinDict["rho1"] = rho1.getVal()
    bukinDict["rho2"] = rho2.getVal()
    func.fitTo(h)
    count += 1
    if count == 100:
        break
massFit.Print()

In [66]:
m.getVal()

115.26009951549382

In [20]:
print(bukinDict)

{'m': 115.25280025975833, 's': 17.77181873371447, 'xi': -0.19519076790363332, 'rho1': -0.19168266726323968, 'rho2': 0.2185122938295418}
